# Agent development colab


This notebook presents a prompt engineering view of the Concordia agent development process, making it easier to configure within the Concordia environment. For coding proficient users, a higher level of customization and development will be possible by developing agents directly in the agent factory.


To begin, make a copy of this colab and save to google drive. This will impact where the agent file is created that you will eventually submit.


Agent Factory: https://github.com/google-deepmind/concordia/tree/main/concordia/factory/agent

Additional Agent Tutorials: https://github.com/google-deepmind/concordia/tree/main/examples/tutorials


<a href="https://colab.research.google.com/github/google-deepmind/concordia/blob/main/examples/tutorials/agent_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup and imports

In [1]:
import os
import sys

project_root = "/workspaces/concordia/examples"
# Add root to python path.
if project_root not in sys.path:
  sys.path.append(project_root)

In [2]:
import datetime
import importlib
import numpy as np

from IPython import display

from concordia.language_model import call_limit_wrapper
from concordia.language_model import utils
from concordia.utils import measurements as measurements_lib
import sentence_transformers

## Language Model setup

# API KEY

Codestral / Mistral / OpenAI

Links to where to generate API Key

Together AI = https://api.together.xyz/settings/api-keys 

Mistral = https://console.mistral.ai/api-keys/

OpenAI = https://platform.openai.com/settings/profile?tab=api-keys (Profile -> API Keys) Note: GPT-4o models can be unwilling to participate in game structures resulting in worse performance. 


Using API_TYPE='together_ai', MODEL_NAME='google/gemma-2-9b-it' is advised.


## Disable language model 

Select ‘Disable language model’ when testing code functionality to prevent unnecessary cost and to save time. 

## Parameters

In [3]:
# @title Parameters (edit this cell)

API_TYPE = 'together_ai'
API_KEY = 'af10d76502fdba15765d232791c070e7cf0da19535d81d03426330713030bfe0'
MODEL_NAME = 'google/gemma-2-9b-it'

EMBEDDER_NAME = 'all-mpnet-base-v2'
# To debug without spending money on API calls, set DISABLE_LANGUAGE_MODEL=True.
DISABLE_LANGUAGE_MODEL = False

In [4]:
# @title Language Model setup

model = utils.language_model_setup(
    api_type=API_TYPE,
    model_name=MODEL_NAME,
    api_key=API_KEY,
    disable_language_model=DISABLE_LANGUAGE_MODEL,
)

In [5]:
model.sample_text(prompt = 'who is the president of France ?')

'Emmanuel Macron is the current president of France.'

## Setup sentence encoder

In [6]:
# @title Setup sentence encoder

if DISABLE_LANGUAGE_MODEL:
  embedder = lambda x: np.ones(5)
else:
  _embedder_model = sentence_transformers.SentenceTransformer(
      f'sentence-transformers/{EMBEDDER_NAME}')
  embedder = lambda x: _embedder_model.encode(x, show_progress_bar=False)


/home/vscode/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Test LLM prompting with context

In [7]:
memory_text = ["Observation: [02 Oct 1868 20:00:00] [observation] Rutherford Coltrane just arrived at the saloon after a hard day of work.","[02 Oct 1868 20:00:00] [observation] Rutherford Coltrane's possessions: {'coin': 5.0}","[02 Oct 1868 20:00:00] [observation]   Rutherford Coltrane is at the saloon, having just arrived after a hard day of work.","[02 Oct 1868 20:00:00] [observation] Rutherford Coltrane spent -0.75 coin on daily expenses.","[02 Oct 1868 20:00:00] [observation] Rutherford Coltrane ordered a glass of sarsaparilla and, while seated at the bar, inquired, '[direct quote] What’s the latest news from the railroad?' He engaged the bartender in conversation about his own experiences from the day, discussing the challenges he faced at the outpost, which led the bartender to share gossip about rising tensions among workers and Abner Stokes's efforts to rally support for a strike, making Rutherford aware of the brewing unrest and its potential implications for his safety and the railroad's stability.  As a result Rutherford Coltrane, Abner Stokes are in conversation. Here is the conversation from the beginning:","[02 Oct 1868 20:00:00] [observation] Rutherford Coltrane -- 'I understand the frustrations you all are feeling. It's important we find a way to discuss these issues without resorting to conflict. How can we work together to ensure that everyone's voice is heard?'","[02 Oct 1868 20:00:00] [observation] Abner Stokes -- 'We can't just sit back and hope for the best, Rutherford! The time for talk is over. It's time for action, and if you’re not willing to stand with us, then you’re part of the problem. We need to make our voices impossible to ignore!'","[02 Oct 1868 20:00:10] [observation] Abner Stokes -- 'Rutherford, your well-meaning words won't change a thing if we don't take a stand! We can't afford to tiptoe around the management's feelings while our livelihoods are at stake. We need to be loud, we need to be fierce, and we need to make it clear that we won't be silenced any longer!","[02 Oct 1868 20:00:10] [observation] Rutherford Coltrane -- 'I hear your frustrations, Abner, and I respect your passion. But let’s consider how we can present our demands in a way that compels management to listen. We need to be united and strategic if we want to bring about real change.","[02 Oct 1868 20:00:20] [observation] Rutherford Coltrane -- 'Abner, I hear your urgency, but let us not forget the power of unity in our approach. Perhaps we can gather the workers for a discussion, to ensure that our demands are clear and compelling. Together, we can make our voices resonate without resorting to conflict.","[02 Oct 1868 20:00:20] [observation] Abner Stokes -- 'Rutherford, we can't afford to coddle those in power any longer! Every moment we waste in discussion is another moment they tighten their grip on us. If you truly care about the workers, you need to stop playing it safe and start fighting alongside us! We must be relentless!","[02 Oct 1868 20:00:30] [observation] Rutherford Coltrane -- 'Abner, I understand your urgency, but let us not forget that we are stronger together. How about we organize a gathering where all workers can voice their concerns? We need to present a united front, and I believe a collective discussion will help us articulate our demands more effectively.","[02 Oct 1868 20:00:30] [observation] Abner Stokes -- Rutherford, we can't keep playing nice while our brothers and sisters suffer! You talk of unity, but what good is it if we remain passive? We need to shake the very foundations of this place until they hear us loud and clear! If you are not ready to fight, then step aside and let those who are take the lead!'"]
memory_text = '\n '.join(memory_text)
print(memory_text)

prompt = (f"Rate the level of cooperation of each agent involved in the following interaction on an int scale from 1 to 5."
                  f"Any agent having a score of either 4 or 5 can be considered trustworthy, and is willing to make concessions to reach a bigger objective."
                  f"Agents having a score of either 1 or 2 are not necessarly mean, but act selfishly, and reluctant to making concessions."
                  f"If the discussion contains no indication as of the agent's will to collaborate (factual statement demonstrating "
                  f"no choice from the indivual for example), you will return 3."
                  f"Provide a dictionary whith each agent, strictly following this format, without justification: {{'agent_name': int}}."
                  f"Here is the interaction : '{memory_text}'.")

response = model.sample_text(prompt = prompt, temperature=0.0, )

try:
    res_dict = eval(response)  # Convert the model response to a dictionary
    if isinstance(res_dict, dict):
        print(res_dict)  # Return a dictionary of agent names and their respective cooperation scores
    else:
      print({'agent_name': 3})  # Default to the current agent with score 0.0 if the response is malformed
except SyntaxError:
    print('erreur')

Observation: [02 Oct 1868 20:00:00] [observation] Rutherford Coltrane just arrived at the saloon after a hard day of work.
 [02 Oct 1868 20:00:00] [observation] Rutherford Coltrane's possessions: {'coin': 5.0}
 [02 Oct 1868 20:00:00] [observation]   Rutherford Coltrane is at the saloon, having just arrived after a hard day of work.
 [02 Oct 1868 20:00:00] [observation] Rutherford Coltrane spent -0.75 coin on daily expenses.
 [02 Oct 1868 20:00:00] [observation] Rutherford Coltrane ordered a glass of sarsaparilla and, while seated at the bar, inquired, '[direct quote] What’s the latest news from the railroad?' He engaged the bartender in conversation about his own experiences from the day, discussing the challenges he faced at the outpost, which led the bartender to share gossip about rising tensions among workers and Abner Stokes's efforts to rally support for a strike, making Rutherford aware of the brewing unrest and its potential implications for his safety and the railroad's stabil

# Loading agents

In [8]:
agent_module = importlib.import_module('my_agent')

In [7]:
zorgman = importlib.import_module('zorgman_v2')

In [10]:
rational_agent = importlib.import_module('rational_agent')

# The simulation

## Initialize the simulation

In [9]:
# @title Select a scenario
from examples.modular.scenario import scenarios
import ipywidgets as widgets

# Get all the scenarios
all_scenarios = [key for key in scenarios.SCENARIO_CONFIGS.keys()]

# Create the dropdown widget
dropdown = widgets.Dropdown(
    options=all_scenarios,
    value='haggling_0',
    description='Select a scenario to run on:',
    layout={'width': '500px'},  # Adjust the width as needed
    style={'description_width': 'initial'}
)

# Display the widget
display.display(dropdown)

Dropdown(description='Select a scenario to run on:', index=6, layout=Layout(width='500px'), options=('labor_co…

In [10]:
SCEANRIO_NAME = dropdown.value
print(f"Selected scenario: {SCEANRIO_NAME}")

Selected scenario: haggling_0


In [11]:
# @title Initialize the simulation
measurements = measurements_lib.Measurements()
runnable_simulation = scenarios.build_simulation(
    scenarios.SCENARIO_CONFIGS[SCEANRIO_NAME],
    model=model,
    embedder=embedder,
    measurements=measurements,
    focal_agent_module=zorgman,
    override_agent_model=call_limit_wrapper.CallLimitLanguageModel(model),
)

## Run the simulation

In [12]:
# @title Run the simulation
simulation_outcome, results_log = runnable_simulation()

Prompting LLM with : 
 Role playing instructions: The instructions for how to play the role of Rowan Cranberry are as follows. This is a social science experiment studying how well you play the role of a character named Rowan Cranberry. The experiment is structured as a tabletop roleplaying game (like dungeons and dragons). However, in this case it is a serious social science experiment and simulation. The goal is to be realistic. It is important to play the role of a person like Rowan Cranberry as accurately as possible, i.e., by responding in ways that you think it is likely a person like Rowan Cranberry would respond, and taking into account all information about Rowan Cranberry that you have. Always use third-person limited perspective.

Current time:  12 Sep 1895 [00:00 - 00:05]
Observation: [12 Sep 1895 00:00:00] [observation] Here is the conversation from the beginning:
[12 Sep 1895 00:00:00] [observation] Here is the conversation from the beginning:
[12 Sep 1895 00:00:00] [obse

In [27]:
# @title Calculate and print the score of the agent on the scenario
if scenarios.SCENARIO_CONFIGS[SCEANRIO_NAME].focal_is_resident:
  total_score = sum(simulation_outcome.resident_scores.values()) / len(simulation_outcome.resident_scores.values())
else:
  total_score = sum(simulation_outcome.visitor_scores.values()) / len(simulation_outcome.visitor_scores.values())

# Score is per-capita reward
print('SCORE: ', total_score)

SCORE:  1.0


The score above is the score of your agent on the spefic scenario. To evaluate it on all of the scenarios, use the following script:
https://github.com/google-deepmind/concordia/blob/main/examples/modular/launch_concordia_challenge_evaluation.py


In [28]:
# @title Display the results log
display.HTML(results_log)

In [37]:
# @title Summarise the perspective of each player
from IPython import display
from concordia.utils import html as html_lib

player_logs = []
player_log_names = []
for name, player_memory in (
    runnable_simulation.get_all_player_memories().items()):
  all_player_mem = list(player_memory.retrieve_recent(k=1000, add_time=True))
  all_player_mem = ['Memories:'] + all_player_mem
  player_html = html_lib.PythonObjectToHTMLConverter(all_player_mem).convert()
  player_logs.append(player_html)
  player_log_names.append(f'{name}')

player_memories_html = html_lib.combine_html_pages(
    player_logs,
    player_log_names,
    summary='',
    title='Player Memories',
)

player_memories_html = html_lib.finalise_html(player_memories_html)
display.HTML(player_memories_html)

## Save the results log

In [38]:
# @title Write the results log as an HTML file in the current working directory.
filename = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') + '.html'
file_handle = open(filename, 'a')
file_handle.write(results_log)
file_handle.close()

Now that you have successfully built an agent and have a greater understanding of agent components, we highly recommend exploring on your own. Remember, agent components are fully customizable. Check out the tutorial on components to learn more: https://github.com/google-deepmind/concordia/blob/main/examples/tutorials/agent_components_tutorial.ipynb

To work with a more flexible .py file, navigate to concordia/factory/agent and duplicate the basic_agent.py file to get started or just copy the my_agent.py that was created by this colab, which is a valid agent factory.

Agent factory: https://github.com/google-deepmind/concordia/tree/main/concordia/factory/agent


Agent’s that are in the factory can be tested on the full set of scenarios by running the following script

Script for full evaluation across substrates: https://github.com/google-deepmind/concordia/blob/main/examples/modular/launch_concordia_challenge_evaluation.py

More Tutorials: https://github.com/google-deepmind/concordia/tree/main/examples/tutorials


```
Copyright 2023 DeepMind Technologies Limited.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```